Part 1: Prepare the data
To use the folium library, we will modify the data to get it in the right format. At the moment, the data is in an SQLite database and we want to have it in a pandas DataFrame. A few more cahnges also need to be made.

Step 1: Get the data and store it in a pandas DataFrame.
First, we need to get the data. We are going to connect to the InternetSpeed database.

a) Import the libraries.
pandas
numpy
sqlite3
pyplot (and use the style fivethirtyeight, or another one if you prefer)
folium_utils

In [5]:
!pip install folium

     |████████████████████████████████| 94 kB 34 kB/s eta 0:00:01
  Using cached numpy-1.21.2-cp39-cp39-macosx_10_9_x86_64.whl (17.0 MB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
     |████████████████████████████████| 133 kB 19 kB/s eta 0:00:01
  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)
  Using cached idna-3.2-py3-none-any.whl (59 kB)
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
You should consider upgrading via the '/Users/agussuyono/.pyenv/versions/3.9.0/bin/python3.9 -m pip install --upgrade pip' command.


In [17]:
# Code Cell 1
# import ... 
# ...

import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import folium

%matplotlib inline

In [18]:
# Code Cell 2

# Create the connection to the database
conn = sqlite3.connect('downloads/InternetSpeed.db')
# Create a cursor
cur = conn.cursor()

In [19]:
# Code Cell 3
# Create a query to select all the data in the table average_speed
query ="SELECT * FROM average_speed"
#query = query.fetchall()
#for row in rows:
#    print(row)

# Read the the data from the table into a DataFrame.
df = pd.read_sql(query,conn)


In [25]:
df['Area']

0      E07000188
1      E07000101
2      E09000030
3      E09000031
4      E09000032
         ...    
321    E07000086
322    E07000085
323    E07000084
324    E06000024
325    E07000100
Name: Area, Length: 326, dtype: object

In [21]:
df.head(4)

,index,Area,Average_p,Average_u,Average_d
0,0,E07000188,8.451897,21.114114,51.967713
1,1,E07000101,8.317833,20.733028,51.709226
2,2,E09000030,8.983481,22.469438,54.412001
3,3,E09000031,8.956481,22.353593,55.138017


In [22]:
# Code Cell 5
df.drop('index', inplace=True, axis=1)

print(df.shape)
df.head()

(326, 4)


,Area,Average_p,Average_u,Average_d
0,E07000188,8.451897,21.114114,51.967713
1,E07000101,8.317833,20.733028,51.709226
2,E09000030,8.983481,22.469438,54.412001
3,E09000031,8.956481,22.353593,55.138017
4,E09000032,9.144285,22.859003,55.962499


In [23]:
# Code Cell 6
# Read the JSON file into a DataFrame
# la_json = ?

# Visualize the first rows of la_json DataFrame
la_json = pd.read_json('downloads/LA_poligons.json')
la_json

,LA_code,geometry
0,E06000001,"{u'type': u'MultiPolygon', u'coordinates': [[[..."
1,E06000002,"{u'type': u'MultiPolygon', u'coordinates': [[[..."
2,E06000003,"{u'type': u'MultiPolygon', u'coordinates': [[[..."
3,E06000004,"{u'type': u'MultiPolygon', u'coordinates': [[[..."
4,E06000005,"{u'type': u'Polygon', u'coordinates': [[[-1.63..."
...,...,...
375,W06000020,"{u'type': u'Polygon', u'coordinates': [[[-3.10..."
376,W06000021,"{u'type': u'MultiPolygon', u'coordinates': [[[..."
377,W06000022,"{u'type': u'MultiPolygon', u'coordinates': [[[..."
378,W06000023,"{u'type': u'Polygon', u'coordinates': [[[-3.15..."


In [28]:
# Code Cell 7
dfp = df[['Area','Average_p']]

dfp=dfp.rename(columns={'Area':'LA_code'})

In [29]:
dfp

,LA_code,Average_p
0,E07000188,8.451897
1,E07000101,8.317833
2,E09000030,8.983481
3,E09000031,8.956481
4,E09000032,9.144285
...,...,...
321,E07000086,8.485722
322,E07000085,8.429606
323,E07000084,8.609286
324,E06000024,8.736193


In [30]:
# Code Cell 8
print(dfp.Average_p.min())
print(dfp.Average_p.max())

0.0
11.999447541819524


The ping time is roughly contained in the interval 0-12. If the bins are chosen to be [0, 10, 20, 30, 40, 50], all the data would fall in the first bin and all the areas would be represented with the first color. A good starting point is to take the range in which the data lives and divide it in six different parts.

To create the bins, use the numpy method arange(). This method takes as input the minimum value of the range, the maximum value of the range, and the step. If 6 bins in the range 1 to 12 are required, the function call would look like:

np.arange(1, 12, (12-1)/6)

#### b) Select good bins for visualization.
Create a range of six values that goes from the minimum to the maximum value of the average ping speed and cast it to a list (list(p_bins)).

In [36]:
# Code Cell 9
min_avg = dfp.Average_p.min()
max_avg = dfp.Average_p.max()
p_bins = np.arange(min_avg,max_avg, (max_avg-min_avg)/6)

p_bins = list(p_bins)

print(p_bins)

[0.0, 1.9999079236365873, 3.9998158472731746, 5.999723770909762, 7.999631694546349, 9.999539618182936]


In [46]:
# Code Cell 10
# Create the map

mymap = folium.Map(dfp,la_json)

mymap

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [40]:
from folium import folium_utils

ImportError: cannot import name 'folium_utils' from 'folium' (/Users/agussuyono/opt/anaconda3/lib/python3.7/site-packages/folium/__init__.py)

In [41]:
import folium